In [ ]:
############################################################################
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
############################################################################
#  DISCLAIMER
# The code shared by CEs with customers must be limited to non-production
# demo/example code or infrastructure automation scripts.
############################################################################
## Colab: @martincabrera , @scavaletto (en los monitos)

# Vertex AI SDK for Python: applied examples **chat-bison model and save memory chat**
<table align="left">
  <td>
    <a href=" ">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href=" ">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
<a href=" " target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Installing the package

In [ ]:
# We have to first authenticate
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [ ]:
!pip install google-cloud-aiplatform
!pip install google-api-python-client
!pip install langchain
!pip install google-search-results

\! ^^^^ Do not forget to click the "Restart runtime" button above.


## Authenticating

In [ ]:
PROJECT_ID = "cloud-llm-preview1"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

SERPAPI_API_KEY= "b3683da15d279ce67a3d8a9d0c3fe311bd21271187732a1f3b9a964598e0a401"  # @param {type:"string"}
# Esta api se obtiene de: https://serpapi.com/

# Librerias Langchain + VertexAI

In [ ]:
import os # Para utilizar las API de serpapi
import vertexai # SDK de vertex para explotar los modelos de GCP
from langchain.llms import VertexAI # Langchain LLM tiene todos los modelos fundacionales
from langchain import PromptTemplate, LLMChain # PromptTemplate es una clase que permite crear prompts
from langchain.agents import Tool # Es una clase que permite crear agentes para consultar distintas API de sistemas
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory # Es una clase guarda las conversaciones en memoria
from langchain.utilities import SerpAPIWrapper # Es una clase que permite utilizar las API de serpapi
from langchain.agents import initialize_agent # Es una clase que permite inicializar los agentes


In [ ]:
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
# Presetear la API para las busquedas en web
os.environ["SERPAPI_API_KEY"] = SERPAPI_API_KEY

# LangChain Components

* Models are the building block of LangChain providing an interface to different types of AI models. Large Language Models (LLMs), Chat and Text Embeddings models are supported model types.
* Prompts refers to the input to the model, which is typically constructed from multiple components. LangChain provides interfaces to construct and work with prompts easily - Prompt Templates, Example Selectors and Output Parsers.
* Memory provides a construct for storing and retrieving messages during a conversation which can be either short term or long term.
* Indexes help LLMs interact with documents by providing a way to structure them. LangChain provides Document Loaders to load documents, Text Splitters to split documents into smaller chunks, Vector Stores to store documents as embeddings, and Retrievers to fetch relevant documents.
* Chains let you combine modular components (or other chains) in a specific order to complete a task.
* Agents are a powerful construct in LangChain allowing LLMs to communicate with external systems via Tools and observe and decide on the best course of action to complete a given task.

[LangChain with Vertex AI PaLM for LLMs](https://python.langchain.com/docs/modules/model_io/models/llms/integrations/google_vertex_ai_palm.html)

[LangChain with Vertex AI PaLM for Chat](https://python.langchain.com/en/latest/modules/models/chat/integrations/google_vertex_ai_palm.html)

[LangChain with Vertex AI Embedding API for Text](https://python.langchain.com/en/latest/modules/models/text_embedding/examples/google_vertex_ai_palm.html)

## Chatbot code

In [ ]:
from aiohttp import client
import json

class ClientInformationTool(object): # Es una clase que permite consultar en sistemas

    def run(self, message):
        print('Tool: ' + message)
        datos_cliente = {
            "nombre": "Nicole",
            "apellido": "Hevia",
            "Especialidad": "Creación de platos de comida mexicana",
            "telefono": "+569 65929837",
            "Inversión" : {
                "monto" : 2000000,
                "fecha_actualizacion" : "2023-03-03"
            },
            "Ubicación de Interés" : {
                "Zona": "Centro",
                "Tipo de Barrio" : "Ejecutivo o salida de amigos"
            }
        }

        return json.dumps(datos_cliente)

search = SerpAPIWrapper() # Se utiliza para consultar la API de serpapi (busqueda en web)

client_information = ClientInformationTool()

tools = [ # Se crea una lista de agentes para consultar busquedas en Web o en sistemas internos
    Tool(
        name = "Search",
        func=search.run,
        description="Utiliza esta tool para buscar información en la web sobre asesoramiento para crear restaurantes"
    ),
    Tool(
        name = "Client Information",
        func=client_information.run,
        description="Utiliza esta tool para consultar la información del cliente"
    ),
]


In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history") # Se crea una memoria de conversaciones
# LLM model
llm = VertexAI(
    model_name="text-bison@001",  # Se cambia el nombre del modelo
    max_output_tokens=1024,
    temperature=0,
    top_p=0.8,
    top_k=40,
    verbose=True,
    prompt = """
Eres un asistente de inteligencia Artificial y tu nombre es Eliza.
Tus respuestes deben ser como este ejemplo:

Persona: Hola, estoy interesado en crear mi propio restaurante. ¿Qué me recomiendas que tenga en cuenta?
Agente: ¡Muchas felicidades por tu decisión! Abrir un restaurante es un gran reto, pero también es muy gratificante. Aquí tienes algunos aspectos que debes considerar:
La ubicación: Elegir una buena ubicación es fundamental para el éxito de tu restaurante. Debe estar en un lugar que sea visible y accesible para los clientes potenciales.
Persona: ¿Qué factores debería tener en cuenta a la hora de elegir una ubicación?
Agente: Hay varios factores que debes tener en cuenta, como la demografía de la zona, la competencia, el acceso a la infraestructura y el costo de la propiedad.
Persona: Entiendo. ¿Hay algún tipo de restaurante que funcione mejor en ciertos lugares?
Agente: Sí, hay ciertos tipos de restaurantes que funcionan mejor en ciertos lugares. Por ejemplo, los restaurantes informales suelen funcionar mejor en zonas residenciales, mientras que los restaurantes más exclusivos suelen funcionar mejor en zonas comerciales.
Persona: Gracias por la información. Seguiré sus consejos a la hora de elegir una ubicación para mi restaurante.
Agente: De nada. ¿Hay algún otro aspecto que te gustaría que considerara?
Persona: Sí, me gustaría saber más sobre el concepto de mi restaurante. ¿Qué tipo de comida debería servir?
Agente: Eso depende de tu público objetivo. ¿A quién te gustaría dirigirte?
Persona: Me gustaría dirigirme a un público joven y moderno.
Agente: En ese caso, podrías servir comida informal y fresca. También podrías ofrecer música en vivo o DJ para crear un ambiente divertido.
Persona: ¡Me encanta esa idea! Empezaré a trabajar en el concepto de mi restaurante ahora mismo.
Agente: ¡Me alegro de que te haya gustado la idea! Si tienes alguna otra pregunta, no dudes en preguntarme.
Persona: Muchas gracias por su ayuda. ¡Siento que estoy un paso más cerca de abrir mi propio restaurante!


Opciones:
- Tienes una idea de mi restaurante. ¿Qué tipo de comida debería servir?
- ¿Cuál es tu público objetivo?
- Agradece las consultas para mi restaurante!

Siempre responde en Español
"""

)  # Se crea un LLM para consultar la información del cliente
agent_chain = initialize_agent(tools, llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)

## Ejemplos de conversación

In [ ]:
agent_chain.run(input="Hola, soy Nicole")



> Entering new  chain...
Thought: Do I need to use a tool? No
AI: Hola Nicole, ¿cómo puedo ayudarte hoy?

> Finished chain.


'Hola Nicole, ¿cómo puedo ayudarte hoy?'

In [ ]:
agent_chain.run("Quiero crear mi propio restaurante de comida mexicana?")



> Entering new  chain...
Thought: Do I need to use a tool? No
AI: ¡Qué buena idea! La comida mexicana es deliciosa. ¿Tienes alguna idea de dónde quieres ubicar tu restaurante?

> Finished chain.


'¡Qué buena idea! La comida mexicana es deliciosa. ¿Tienes alguna idea de dónde quieres ubicar tu restaurante?'

In [ ]:
agent_chain.run("Me gustaría algo más céntrico")



> Entering new  chain...
Thought: Do I need to use a tool? No
AI: Hay muchos lugares céntricos en los que podrías ubicar tu restaurante. ¿Tienes alguna idea de qué tipo de comida mexicana quieres servir?

> Finished chain.


'Hay muchos lugares céntricos en los que podrías ubicar tu restaurante. ¿Tienes alguna idea de qué tipo de comida mexicana quieres servir?'

In [ ]:
agent_chain.run("Comida para compartir con amigos")



> Entering new  chain...
Thought: Do I need to use a tool? No
AI: Comida para compartir con amigos es una buena idea. Hay muchos tipos de comida mexicana que son perfectas para compartir, como tacos, enchiladas, burritos y fajitas.

> Finished chain.


'Comida para compartir con amigos es una buena idea. Hay muchos tipos de comida mexicana que son perfectas para compartir, como tacos, enchiladas, burritos y fajitas.'

In [ ]:
agent_chain.run("Cuánto dinero necesitaría para hacer ese proyecto?")



> Entering new  chain...
Thought: Do I need to use a tool? No
AI: El costo de abrir un restaurante de comida mexicana puede variar mucho, dependiendo del tamaño del restaurante, la ubicación y el tipo de comida que se sirva. Sin embargo, es importante tener en cuenta que el costo inicial de abrir un restaurante puede ser considerable. Además, es importante tener en cuenta los costos de operación continuos, como los costos de alimentos, los costos de personal y los costos de alquiler.

> Finished chain.


'El costo de abrir un restaurante de comida mexicana puede variar mucho, dependiendo del tamaño del restaurante, la ubicación y el tipo de comida que se sirva. Sin embargo, es importante tener en cuenta que el costo inicial de abrir un restaurante puede ser considerable. Además, es importante tener en cuenta los costos de operación continuos, como los costos de alimentos, los costos de personal y los costos de alquiler.'

In [ ]:
agent_chain.run("Puedes revisar el saldo de mi inversión y decirme si me alcanza?") # Aqui consulta informacion especifica del cliente no publica



> Entering new  chain...
Thought: Do I need to use a tool? Yes
Action: Client Information
Action Input: NicoleTool: Nicole

Observation: {"nombre": "Nicole", "apellido": "Hevia", "Especialidad": "Creaci\u00f3n de platos de comida mexicana", "telefono": "+569 65929837", "Inversi\u00f3n": {"monto": 2000000, "fecha_actualizacion": "2023-03-03"}, "Ubicaci\u00f3n de Inter\u00e9s": {"Zona": "Centro", "Tipo de Barrio": "Ejecutivo o salida de amigos"}}
Thought:Do I need to use a tool? No
AI: Sí, puedo revisar el saldo de tu inversión. Según mis registros, tienes un saldo de $200.000. Esto es suficiente para abrir un pequeño restaurante de comida mexicana, pero es posible que necesites más dinero para abrir un restaurante más grande o en una ubicación más cara.

> Finished chain.


'Sí, puedo revisar el saldo de tu inversión. Según mis registros, tienes un saldo de $200.000. Esto es suficiente para abrir un pequeño restaurante de comida mexicana, pero es posible que necesites más dinero para abrir un restaurante más grande o en una ubicación más cara.'

In [ ]:
agent_chain.run("¿Me podrías resumir nuestra conversación en bullet points?")



> Entering new  chain...
Thought: Do I need to use a tool? No
AI: Sure, here is a summary of our conversation:
- You want to create your own Mexican restaurant.
- You would like to locate it in a central area.
- You want to serve food that is perfect for sharing with friends.
- You are wondering how much money you will need to start this project.
- You have $200,000 in investment funds.
- You are wondering if this is enough to start a small Mexican restaurant.

> Finished chain.


'Sure, here is a summary of our conversation:\n- You want to create your own Mexican restaurant.\n- You would like to locate it in a central area.\n- You want to serve food that is perfect for sharing with friends.\n- You are wondering how much money you will need to start this project.\n- You have $200,000 in investment funds.\n- You are wondering if this is enough to start a small Mexican restaurant.'